In [19]:
# Load vector database that was persisted earlier and check collection count in it
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
persist_directory = 'vector_db/chroma/'
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
print(vectordb._collection.count())

Using embedded DuckDB with persistence: data will be stored in: vector_db/chroma/


AttributeError: 'Embedding' object has no attribute 'embed_documents'

In [4]:
question = "what is Quasarion?"
docs = vectordb.similarity_search(question,k=3)
len(docs)
for res in docs:
    print (res.page_content)
    print ('============')

NoIndexException: Index not found, please create an instance before querying

In [12]:
from transformers import AutoTokenizer, DistilBertForQuestionAnswering, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased-distilled-squad')
model = AutoModelForQuestionAnswering.from_pretrained('distilbert-base-cased-distilled-squad')

# Load model directly

pipe = pipeline("question-answering", 
                model=model, 
                tokenizer=tokenizer,     
                temperature=0.2,
                max_new_tokens=512)

llm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs={"temperature": 0.2, "max_length": 1024},
)
model.distilbert.embeddings.word_embeddings.

Embedding(28996, 768, padding_idx=0)

In [14]:
model.distilbert.embeddings.word_embeddings

Embedding(28996, 768, padding_idx=0)

In [2]:
# from langchain.chains.question_answering import load_qa_chain
# chain = load_qa_chain(llm, chain_type="stuff",verbose=True)
import torch
query = "What is Quasarion ?"
matching_docs = vectordb.max_marginal_relevance_search(query)

context = ''
for i in matching_docs:
    context += " " + i.page_content

inputs = tokenizer(question, context[:512], return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()
if answer_start_index > answer_end_index:
    answer_end_index, answer_start_index = answer_start_index, answer_end_index
print (answer_start_index, answer_end_index)
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
print (predict_answer_tokens)
tokenizer.decode(predict_answer_tokens)

NameError: name 'vectordb' is not defined

In [2]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"

inputs = tokenizer(question, text, return_tensors='pt')

start_positions = torch.tensor([1])
end_positions = torch.tensor([3])

outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
loss = outputs.loss

start_scores = outputs.start_logits
end_scores = outputs.end_logits 
start_scores, end_scores

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(tensor([[ 0.2069, -0.0259,  0.3412,  0.0153,  0.4669,  0.2928,  0.0926,  0.0060,
           0.4558,  0.2290,  0.2784,  0.2105,  0.1995,  0.0926]],
        grad_fn=<CloneBackward0>),
 tensor([[-0.3415,  0.0370, -0.1975, -0.4639, -0.0913,  0.3339,  0.2071, -0.4206,
          -0.1138, -0.1672, -0.3959, -0.2285, -0.1096,  0.2071]],
        grad_fn=<CloneBackward0>))